### Extract and clean time series data

In [1]:
from utils import fetch_log_returns
training_dataset = fetch_log_returns(start = '1995-01-01', end ='1995-12-31')

[*********************100%***********************]  5 of 5 completed


In [2]:
training_dataset.head()

,sp500,dax,ftse,nikkei,ibex
Date,,,,,
1995-01-05 00:00:00+00:00,-0.000803,-0.010397,-0.006345,-0.003457,-0.019565
1995-01-06 00:00:00+00:00,0.000738,0.003280,0.010726,-0.004939,0.000000
1995-01-09 00:00:00+00:00,0.000326,-0.002330,-0.003006,-0.003826,-0.014346
1995-01-10 00:00:00+00:00,0.001843,0.004213,0.001504,0.002903,-0.005274
1995-01-11 00:00:00+00:00,-0.000043,-0.000155,-0.003601,0.002408,-0.010125


### Define the problem instance and model parameters

In [12]:
from model_params import LAMBDA_1, LAMBDA_2, LAMBDA_3, NLAYERS, NSHOTS, NUM_ASSETS, SIGMA_TARGET, TWO_QUBIT_GATES, K, N, RISK_FREE_RATE


### Build the circuit Ansatz (without angle embedding)

In [4]:
from ansatz import build_hardware_efficient_ansatz
from ansatz import compute_number_of_params
ansatz = build_hardware_efficient_ansatz(num_qubits=N, num_layers=NLAYERS, params=[0]*compute_number_of_params(N,NLAYERS))
print('Hardware efficient circuit ansatz')
print()
print(ansatz.draw())


Hardware efficient circuit ansatz

q0: ─U2─U2─o───────U1─U2─o───────U1─M─
q1: ─U2─U2─X─o─────U1─U2─X─o─────U1─M─
q2: ─U2─U2───X─o───U1─U2───X─o───U1─M─
q3: ─U2─U2─────X─o─U1─U2─────X─o─U1─M─
q4: ─U2─U2───────X─U1─U2───────X─U1─M─


### Build the Portfolio Optimization Hamiltonian and run the optimization process


In [5]:
import numpy as np
from qibo.optimizers import optimize
from cost_function import compute_total_energy

# Optimize starting from a random guess for the variational parameters
initial_params = np.random.uniform(0, 2*np.pi, compute_number_of_params(N,NLAYERS))

# perform optimization
best, optimal_params, extra = optimize(compute_total_energy, initial_params, args=(training_dataset))

# set final solution to circuit instance
print('Ground state energy:', best)
print()
print('Optimal parameters', optimal_params)
print()
print('Optimization process info', extra)

[Qibo 0.2.8|INFO|2024-10-03 20:24:43]: Using numpy backend on /CPU:0


Ground state energy: 156.30049210320266

Optimal parameters [3.28747628 5.394333   1.78260901 1.96569609 4.92335797 4.83797578
 4.5243877  4.78963462 2.09453765 4.65253642 1.65320103 1.11342001
 2.80865349 2.48452671 3.28107242 3.87547935 0.90789148 3.99348565
 3.17187684 4.44027871 1.17560957 2.0704244  5.35520415 2.51829988
 4.76580599 4.57446476 4.42964952 0.05158818 2.69127382 2.22238238
 3.75670278 4.79705541 0.99956735 5.96246604 3.21909487 5.63548272
 0.33550344 5.88756038 0.92545495 6.2039591 ]

Optimization process info  message: Optimization terminated successfully.
 success: True
  status: 0
     fun: 156.30049210320266
       x: [ 3.287e+00  5.394e+00 ...  9.255e-01  6.204e+00]
     nit: 1
   direc: [[ 1.000e+00  0.000e+00 ...  0.000e+00  0.000e+00]
           [ 0.000e+00  1.000e+00 ...  0.000e+00  0.000e+00]
           ...
           [ 0.000e+00  0.000e+00 ...  1.000e+00  0.000e+00]
           [ 0.000e+00  0.000e+00 ...  0.000e+00  1.000e+00]]
    nfev: 121


### Results parsing


In [35]:
def get_minimum_energy(portfolios: pd.DataFrame) -> float:
    energies = []
    for data in portfolios.values():
        energies.append(data['energy'])
    return min(energies)
get_minimum_energy(optimal_binary_portfolios)

100.0625

In [49]:
from numpy import number
from qibo.result import CircuitResult
def get_max_prob(result: CircuitResult, nshots: int = NSHOTS) -> float:
    number_of_times = result.frequencies().values()
    probs = [freq/nshots for freq in number_of_times]
    return max(probs)

In [50]:
result = ansatz(nshots=100)


0.08

In [53]:
from qibo.models import Circuit
from cost_function import compute_cost_function
from utils import string_to_int_list
import pandas as pd
TOLERANCE = 0.5

def get_optimal_binary_portfolios(ansatz: Circuit, dataset: pd.DataFrame, nshots: int = NSHOTS, tolerance: int = TOLERANCE) -> dict:
    result = ansatz(nshots=nshots)
    optimal_portfolios = {}
    for bit_string, stat_freq in result.frequencies().items():
        if (get_max_prob(result) - stat_freq/nshots) < tolerance:
            optimal_portfolios[bit_string] = {'stat_freq': stat_freq/nshots, 'energy': compute_cost_function(dataset, string_to_int_list(bit_string))}
    return optimal_portfolios


optimized_ansatz = ansatz.set_parameters(optimal_params)
optimal_binary_portfolios = get_optimal_binary_portfolios(ansatz, training_dataset)


In [54]:
optimal_binary_portfolios

{'00000': {'stat_freq': 0.04, 'energy': 264.033506669926},
 '00001': {'stat_freq': 0.01, 'energy': 224.9904736273165},
 '00010': {'stat_freq': 0.03, 'energy': 224.97217237979967},
 '00101': {'stat_freq': 0.02, 'energy': 189.0766958797133},
 '00111': {'stat_freq': 0.01, 'energy': 156.26536158961062},
 '01000': {'stat_freq': 0.11, 'energy': 224.98184885674965},
 '01001': {'stat_freq': 0.02, 'energy': 189.0638158141494},
 '01010': {'stat_freq': 0.06, 'energy': 189.04551456665496},
 '01100': {'stat_freq': 0.04, 'energy': 189.06807110915688},
 '01110': {'stat_freq': 0.03, 'energy': 156.2567368190767},
 '10000': {'stat_freq': 0.04, 'energy': 225.0073028934075},
 '10001': {'stat_freq': 0.05, 'energy': 189.08926985080106},
 '10010': {'stat_freq': 0.06, 'energy': 189.07096860329156},
 '10011': {'stat_freq': 0.01, 'energy': 156.27793556069423},
 '10100': {'stat_freq': 0.06, 'energy': 189.09352514580488},
 '10101': {'stat_freq': 0.04, 'energy': 156.30049210320266},
 '10110': {'stat_freq': 0.04, '

In [56]:
def get_asset_weights_binary(assets, ordered_bitstring, num_qubit_per_asset = K):
    weights = [ordered_bitstring[i:i+num_qubit_per_asset] for i in range(0, len(ordered_bitstring),num_qubit_per_asset)]
    return dict(zip(assets,weights))

optimal_binary_portfolio=list(optimal_binary_portfolios.keys())[0]

binary_asset_weights = get_asset_weights_binary(training_dataset.columns, optimal_binary_portfolio)
binary_asset_weights

{'sp500': '0', 'dax': '0', 'ftse': '0', 'nikkei': '0', 'ibex': '0'}

In [9]:

def get_asset_weight_decimal(asset_bit_string):
    w = 0
    for k,bit in enumerate(asset_bit_string):
       w += 2**(k-1)*int(bit)*1/(2**len(asset_bit_string)) # discretization !
    return w  

get_asset_weight_decimal(binary_asset_weights['nikkei'])

0.0

In [10]:
portfolio = {}
for asset, w in binary_asset_weights.items():
    portfolio[asset] = get_asset_weight_decimal(w)
portfolio

{'sp500': 0.0, 'dax': 0.0, 'ftse': 0.0, 'nikkei': 0.0, 'ibex': 0.0}

### Calculating portfolio metrics

In [13]:
def get_portfolio_metrics(portfolio: dict, dataset: pd.DataFrame, r: float = RISK_FREE_RATE):
    import numpy as np
    normalized_weights = list(portfolio.values()) / np.sum(list(portfolio.values()))


    # Calculate the expected log returns, and add them to the `returns_array`.
    annualized_ret_portfolio = np.sum((dataset.mean() * normalized_weights) * 252)


    # Calculate the volatility, and add them to the `volatility_array`.
    annualized_vol_portfolio = np.sqrt(
        np.dot(normalized_weights.T, np.dot(dataset.cov() * 252, normalized_weights))
    )
    annualized_ret_portfolio,annualized_vol_portfolio

    # Calculate the Sharpe Ratio and Add it to the `sharpe_ratio_array`.
    sharpe_ratio= (annualized_ret_portfolio-r)/annualized_vol_portfolio

    # Let's create our "Master Data Frame", with the weights, the returns, the volatility, and the Sharpe Ratio
    return {'Returns':annualized_ret_portfolio, 'Volatility':annualized_vol_portfolio, 'Sharpe Ratio':sharpe_ratio, 'Normalized Weights':normalized_weights}
get_portfolio_metrics(portfolio, training_dataset, r=0.1)

/tmp/ipykernel_1297365/2935572182.py:3: RuntimeWarning: invalid value encountered in divide
  normalized_weights = list(portfolio.values()) / np.sum(list(portfolio.values()))


{'Returns': 0.0,
 'Volatility': nan,
 'Sharpe Ratio': nan,
 'Normalized Weights': array([nan, nan, nan, nan, nan])}